### Just so that you can feed an initial condition and play quickly check the trajectory...

In [20]:
using DifferentialEquations
using Plots
using LinearAlgebra
import ForwardDiff
import DiffResults
using AstrodynamicsBase
# import joptimise
using Printf
using DataFrames
using JSON
using CSV
plotly()

Plots.PlotlyBackend()

In [37]:
include("../../julia-r3bp/R3BP/src/R3BP.jl")
include("../src/SailorMoon.jl")   # relative path to main file of module

┌ Warning: Replacing docs for `Main.SailorMoon.integrate_rk4 :: Union{Tuple{ODEProblem, Real}, Tuple{ODEProblem, Real, Union{Nothing, Vector}}, Tuple{ODEProblem, Real, Union{Nothing, Vector}, Bool}, Tuple{ODEProblem, Real, Union{Nothing, Vector}, Bool, Bool}}` in module `Main.SailorMoon`
└ @ Base.Docs docs\Docs.jl:240


Main.SailorMoon

In [38]:
#### callback functions ###################
function terminate_condition(u,t,int)
    # Hit earth
    cond1 = sqrt((u[1] + param3b.mu2) ^2 + u[2]^2 + u[3]^2) - (1000 / param3b.lstar)
    # Hit moon
    cond2 = sqrt((u[1] - (1-param3b.mu2)) ^2 + u[2]^2 + u[3]^2) - (870 / param3b.lstar)
    return - cond1 * cond2
end

# store the apoapsis value
function apoapsis_cond(u,t,int)
    # condition 1: SC is sufficiently far from the moon
    r = sqrt((u[1] - (1-param3b.mu2))^2 + u[2]^2 + u[3]^2) # SC-Moon distance
    moon_soi = 5000 / param3b.lstar # define "sphere of influence"
    
    if sqrt(u[1]^2 + u[2]^2 + u[3]^2) > 1.5 
        # condition 2: dot product of velocity and position is zero
        return dot((u[1:3] + [param3b.mu2, 0.0, 0.0]), u[4:6])
    else
        return NaN
    end
end

# store the periapsis value and terminate
function periapsis_cond(u,t,int)
    r = sqrt((u[1] + param3b.mu2)^2 + u[2]^2 + u[3]^2)  # SC-earth distance
    earth_leo_ub = (6357 + 1500) / param3b.lstar
    earth_leo_lb = 2000 / param3b.lstar
    
    if earth_leo_lb < r < earth_leo_ub
        return dot((u[1:3] + [param3b.mu2, 0.0, 0.0]), u[4:6])
    else 
        return NaN
    end
end

# generalized aps condition 
function aps_cond(u,t,int)
    pos = u[1:3] + [param3b.mu2, 0.0, 0.0]
    return dot(pos, u[4:6]) 
end

# perilune
function perilune_cond(u,t,int)
    r = sqrt((u[1] - (1 - param3b.mu2))^2 + u[2]^2 + u[3]^2)  # SC-Moon distance
    moon_soi = 66000 / param3b.lstar
    
    if r < moon_soi && -t > (10*86400 / param3b.tstar)
        return dot((u[1:3] - [1 - param3b.mu2, 0.0, 0.0]), u[4:6])
    else 
        return NaN
    end
end


# affect!
terminate_affect!(int) = terminate!(int)
no_affect!(int) = NaN



no_affect! (generic function with 1 method)

In [39]:
function rhs_bcr4bp_thrust!(du,u,p,t)
    # unpack arguments
    μ, μ_3, t0, a_s, ω_s, τ, θ, β, mdot, tmax = p
    # decompose state
    x, y, z, vx, vy, vz, mass = u

    # calculate radii
    r1 = sqrt( (x+μ)^2 + y^2 + z^2 )
    r2 = sqrt( (x-1+μ)^2 + y^2 + z^2 )

    # sun position
    xs = a_s*cos(ω_s*t + t0)
    ys = a_s*sin(ω_s*t + t0)
    zs = 0.0
    r3 = sqrt( (x-xs)^2 + (y-ys)^2 + (z-zs)^2 )

    # compute delta-V vector
#     dir_v = dv_lvlh2inertial(u[1:6], [τ, θ, β])
    dir_v = [0.0, 0.0, 0.0]
    ts = tmax*dir_v

    # position-state derivative
    du[1] = vx
    du[2] = vy
    du[3] = vz

    # velocity derivatives: Earth, moon, sun
    du[4] =  2*vy + x - ((1-μ)/r1^3)*(μ+x)  + (μ/r2^3)*(1-μ-x)  + ( -(μ_3/r3^3)*(x-xs) - (μ_3/a_s^3)*xs ) + ts[1]/mass
    du[5] = -2*vx + y - ((1-μ)/r1^3)*y      - (μ/r2^3)*y        + ( -(μ_3/r3^3)*(y-ys) - (μ_3/a_s^3)*ys ) + ts[2]/mass
    du[6] =           - ((1-μ)/r1^3)*z      - (μ/r2^3)*z        + ( -(μ_3/r3^3)*(z)    - (μ_3/a_s^3)*zs ) + ts[3]/mass
    
#     println(r1, " ", r2, " ", r3)
#     println(((1-μ)/r1^3)*(μ+x), " ", (μ/r2^3)*(1-μ-x), " ",  -(μ_3/r3^3)*(x-xs) )

    # mass derivative
    du[7] = -mdot*τ
end

rhs_bcr4bp_thrust! (generic function with 1 method)

In [40]:
function plot_circle(radius, x, y, n=50)
    circle = zeros(2,n)
    thetas = LinRange(0.0, 2π, n)
    for i = 1:n
        circle[1,i] = radius*cos(thetas[i]) + x
        circle[2,i] = radius*sin(thetas[i]) + y
    end
    return circle
end

plot_circle (generic function with 2 methods)

In [41]:
param3b = SailorMoon.dyanmics_parameters()
lps = SailorMoon.lagrange_points(param3b.mu2)

## set up of initial condition (Lyapunov orbit)
lp = 2
Az_km = 1200.0
println("Halo guess Az_km: $Az_km")
northsouth = 3   # 1 or 3
guess0 = R3BP.halo_analytical_construct(param3b.mu2, lp, Az_km, param3b.lstar, northsouth)
res = R3BP.ssdc_periodic_xzplane([param3b.mu2,], guess0.x0, guess0.period, fix="period")

x0_stm = vcat(res.x0, reshape(I(6), (6^2,)))[:]
prob_cr3bp_stm = ODEProblem(R3BP.rhs_cr3bp_svstm!, x0_stm, res.period, (param3b.mu2))
sol = solve(prob_cr3bp_stm, Tsit5(), reltol=1e-12, abstol=1e-12)#, saveat=LinRange(0, period, n+1))
monodromy = R3BP.get_stm(sol, 6)   # get monodromy matrix
ys0 = R3BP.get_eigenvector(monodromy, true, 1) # monodromy eigenvector

ϵr = 1e-6;   
ϵv = 1e-6;   

Halo guess Az_km: 1200.0
Linear stability ν = 618.7618470919092


In [45]:
### Change initial condition here ###

ϕ0    = 5.27787565803085 #5.280569324590653 
θsf   = 0.125663706 #0.12987080307120433
tof_bck  = 18.8977945731596 # 18.8983242715549  

θmf = π - θsf                
# arrival LPO object
LPOArrival = SailorMoon.CR3BPLPO2(
    res.x0, res.period, ys0, prob_cr3bp_stm, ϵr, ϵv, Tsit5(), 1e-12, 1e-12, 0.005
);

xf = vcat(SailorMoon.set_terminal_state2(ϕ0, θmf, param3b, LPOArrival), 1.0)
println(res.period)

3.410360993933259


In [43]:
tspan = [0.0, -tof_bck]
params=[param3b.mu2, param3b.mus, θsf, param3b.as, param3b.oms, 0.0, 0.0, 0.0, 0.0, 0.0]
prob = ODEProblem(rhs_bcr4bp_thrust!, xf, tspan, params)
sol = solve(prob, Tsit5(), reltol=1e-12, abstol=1e-12);


In [44]:
pcart = plot(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4)
plot!(pcart, Array(sol)[1,:], Array(sol)[2,:], color=:blue, linewidth=1.0, label="sol", linestyle=:solid)

moon = plot_circle(1738/param3b.lstar, 1-param3b.mu2, 0.0)
earth = plot_circle(6375/param3b.lstar, -param3b.mu2, 0.0)
moon_soi = plot_circle(66000/param3b.lstar, 1-param3b.mu2, 0.0)
leo_lb = plot_circle((2000)/param3b.lstar, -param3b.mu2, 0.0)
leo_ub = plot_circle((6375+1500)/param3b.lstar, -param3b.mu2, 0.0)
plot!(pcart, leo_lb[1,:], leo_lb[2,:], c=:black, lw=1.0, label="LEO lb")
plot!(pcart, leo_ub[1,:], leo_ub[2,:], c=:black, lw=1.0, label="LEO ub")
plot!(pcart, earth[1,:], earth[2,:], c=:green, lw=1.0, label="earth")
plot!(pcart, moon[1,:], moon[2,:], c=:orange, lw=1.0, label="moon")
plot!(pcart, moon_soi[1,:], moon_soi[2,:], c=:black, lw=1.0, label="moon soi")

pcart